In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib notebook

In [2]:
import lightgbm as lgb

from LGBMSelGB import LGBMSelGB
from utils import compare_model_error, load_data

In [3]:
base_dir = "../datasets/istella-short/sample"
train_file = base_dir + "/train.txt"
valid_file = base_dir + "/vali.txt"
test_file = base_dir + "/test.txt"

In [4]:
train_data, train_labels, train_query_lens = load_data(train_file)
print('Training set loaded')

Training set loaded


In [5]:
valid_data, valid_labels, valid_query_lens = load_data(valid_file)
print('Validation set loaded')

Validation set loaded


In [6]:
test_data, test_labels, test_query_lens = load_data(test_file)
print('Testing set loaded')

Testing set loaded


In [7]:
train_set = lgb.Dataset(train_data, label=train_labels, group=train_query_lens)
eval_set = [(train_data, train_labels),
            (valid_data, valid_labels),
            (test_data, test_labels)]
eval_group = [train_query_lens, valid_query_lens, test_query_lens]
eval_names = ['train', 'valid', 'test']
valid_sets = []
for i, valid_data in enumerate(eval_set):
    ds = lgb.Dataset(valid_data[0], valid_data[1], group=eval_group[i], reference=train_set)
    valid_sets.append(ds)

params = {
    'objective': 'lambdarank',
    'max_position': 10,
    'learning_rate': 0.05,
    'num_leaves': 64,
    'metric': ['ndcg'],
    'ndcg_eval_at': 10
}
lgb_info = {}

In [8]:
base_model = lgb.train(params, train_set, num_boost_round=250,
                       #valid_sets=valid_sets, valid_names=eval_names,
                       verbose_eval=10, evals_result=lgb_info)

In [47]:
selgb_base = LGBMSelGB(n_estimators=250, n_iter_sample=10, max_p=0.2, method='fixed')
selgb_base.fit(train_data, train_labels, train_query_lens, params,
           #eval_set=eval_set, eval_group=eval_group, eval_names=eval_names,
           verbose=10)

TypeError: 'NoneType' object is not iterable